In [1]:
#Ładuję potrzebne biblioteki
import re
from datetime import datetime
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import os, sys
import glob
from matplotlib.backends.backend_pdf import PdfPages
import time

In [2]:
# Tworzymy listę i dwa puste DF
listOfFiles = []
DFL = pd.DataFrame()
DFM = pd.DataFrame() 

In [3]:
# Funkcja pobiera listę ścieżek
def get_filenames():
    Tk().withdraw()
    print("Initializing Dialogue... \nPlease select a file.")
    tk_filenames = filedialog.askdirectory()
    tempDir = tk_filenames
    return tempDir


choosen_dir = get_filenames()
# glob zbiera listę plików 
os.chdir(choosen_dir)
for file in glob.glob("*.csv"):
    listOfFiles.append(file)

# Do wcześniej stworzonych DF dodajemy kolejno pliki w zależności od tego na jaką literę plik się zaczyna.
for file in listOfFiles:
    if file.startswith('L'):
        Table = pd.read_csv(file, sep=',')
        DF1 = pd.DataFrame(Table)
        DFL = DFL.append(DF1, sort=False)
    elif file.startswith('M'):
        Table2 = pd.read_csv(file, sep=',')
        DF2 = pd.DataFrame(Table2)
        DFM = DFM.append(DF2, sort=False)

Initializing Dialogue... 
Please select a file.


In [4]:
# Deklaruje listę kolumn którę będą mi potrzebne
DFL['NMIOT'] = 1
cols1 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','NROJM1','LSUT1', 'NRMAM1', 'NOJMIO']
DFL = DFL[cols1]

In [5]:
# Zmieniam nazwy kolumn tak by odpowiadały kolumnon z DFL
DFM['LSUT1'] = np.nan  
DFM = DFM.rename(columns={'LDUR': 'LDUR1'})
DFM = DFM.rename(columns={'LDOP': 'LDUR'})
DFM = DFM.rename(columns={'LDUR1': 'LDOP'})
DFM = DFM.rename(columns={'RASAOJ': 'LRASAOJ'})
DFM = DFM.rename(columns={'NRO': 'NROJM1'})
DFM = DFM.rename(columns={'LREJ': 'LREJ1'})
DFM = DFM.rename(columns={'LWOJ': 'LWOJ1'})
# Deklaruje listę kolumn którę będą mi potrzebne
cols2 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','LSUT1','NROJM1']
DFM = DFM[cols2]

In [6]:
# Rozdzielam sutki prawe od lewych
DFL['PSUT'] = DFL['LSUT1'].astype(str).str[1]
DFL['LSUT1'] = DFL['LSUT1'].astype(str).str[0]

In [7]:
# Convertuje liczby sutków do inta
convert_dict = {'LSUT1': int,
                'PSUT': int
               }
  
DFL = DFL.astype(convert_dict)

In [8]:
# Liczę sumę sutków prawych i lewych
DFL['ISUT'] = DFL['PSUT'] + DFL['LSUT1']

In [9]:
# Wprowadzamy przedział czasowy z jakiego będą pobieranie dane
date1 = input('Date range from YYYY-MM-DD') 
date2 = input('Date range to YYYY-MM-DD')

Date range from YYYY-MM-DD 2019-12-12
Date range to YYYY-MM-DD 2020-12-11


In [10]:
# Funkcja zamienia format kolumn na format daty
class ToDate():

    def change_to_date(self, a, b):
        a[b] = pd.to_datetime(a[b])


dat = ToDate()
dat.change_to_date(DFL, 'LDOP')
dat.change_to_date(DFM, 'LDOP')
dat.change_to_date(DFL, 'LDUR')
dat.change_to_date(DFM, 'LDUR')

In [11]:
# W DF zostawaimy tylko dane z wcześniej zadeklarowanego przedziału czasowego
DFL = DFL[(DFL['LDOP'] >= date1) & (DFL['LDOP'] <= date2)]
DFM = DFM[(DFM['LDOP'] >= date1) & (DFM['LDOP'] <= date2)]

In [12]:
# Wyliczamy:
# Datę pierwszego oprosienia
DFL['POP'] = DFL['LDOP'] - DFL['LDUR']
# Okres międzymiotu
DFM['OMM'] = DFM['LDOP'] - DFM['LDUR']
# Zamieniam ww wartości na dni
DFL['POP'] = DFL['POP'] / np.timedelta64(1, 'D')
DFM['OMM'] = DFM['OMM'] / np.timedelta64(1, 'D')

In [13]:
# W zmiennej DF3 łączymy poprzednie DF
DF3 = DFL.append(DFM)

In [14]:
DF3 = DF3.rename(columns={'LDUR': 'DURL'})
DF3 = DF3.rename(columns={'LDOP': 'DURM'})

In [15]:
# Wyliczamy ile prosiąt nie dożyło 21 dnia życia
DF3['STRATY'] = DF3['LIL11'] - DF3['LIL21']
# Wyliczamy wartość % strat
DF3['ST21'] = (DF3['STRATY'] / DF3['LIL11']) * 100

In [17]:
# Funkcja tworzy DF z określonymi już kolumnami
def createDT(name):
    temp = pd.DataFrame(columns = ['Rasa','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = name
    return temp

# Funkcja sortuje dane po odpowiedniej rasie
def sortedDT(DT, col, number):
    newDT = DT[DT[col] == number]
    return newDT
# Funkcja sortuje dane po rasie matki i rasie ojca
def sorted2DT(DT,number1,number2):
    newDT = DT[(DT.LRASA1 == number1) & (DT.LRASAOJ == number2)]
    return newDT

def months(DT, col, col2):
    newDT = DT[DT[col] != DT[col2]]
    return newDT

# Funkcja wylicza wszystkie niezbędne wartości i przypisuje je do DF
def all_count(DF3, new):
    new['liczba loch'] = len(DF3.drop_duplicates(subset='LPROS1', keep="first").copy())
    new['liczba ocenionych miotów ogółem'] = len(DF3)
    new['liczba ocenionych miotów pierw.'] = len(DF3[DF3['NMIOT']==1])
    new['Liczba prosiąt urodzonych w miocie'] = np.around(DF3['LIL11'].mean(), decimals=2)
    new['Liczba prosiąt w 21 dniu życia'] = np.around(DF3['LIL21'].mean(), decimals=2)
    new['Straty prosiąt do 21 dnia życia'] = np.around(DF3['ST21'].mean(), decimals=2)
    new['Wiek w dniu pierwszego oprosienia(dni)'] = np.around(DF3['POP'].mean())
    new['Okres miedzymiotu(dni)'] = np.around(DF3['OMM'].mean())
    new['Liczba sutków lochy'] = np.around(DF3['ISUT'].mean(), decimals=2)

    return new

In [18]:
WBPxWBP = createDT('WBP x WBP')
wbpXwbp = sorted2DT(DF3,10,10)
all_count(wbpXwbp,WBPxWBP)

WBPxPBZ = createDT('WBP x PBZ')
wbpXpbz = sorted2DT(DF3,10,20).copy()
all_count(wbpXpbz,WBPxPBZ)

PBZxPBZ = createDT('PBZ x PBZ')
pbzXpbz = sorted2DT(DF3,20,20).copy()
all_count(pbzXpbz,PBZxPBZ)

PBZxWBP = createDT('PBZ x WBP')
pbzXwbp = sorted2DT(DF3,20,10).copy()
all_count(pbzXwbp,PBZxWBP)

DUROCxDUROC = createDT('Duroc x Duroc')
durocXduroc = sorted2DT(DF3,70,70).copy()
all_count(durocXduroc,DUROCxDUROC)

DUROCxPIETRAIN = createDT('Duroc x Pietrain')
durocXpietrain = sorted2DT(DF3,70,80).copy()
all_count(durocXpietrain,DUROCxPIETRAIN)

PIETRAINxPIETRAIN = createDT('Pietrain x Pietrain')
pietrainXpietrain = sorted2DT(DF3,80,80).copy()
all_count(pietrainXpietrain,PIETRAINxPIETRAIN)

PIETRAINxDUROC = createDT('Pietrain X Duroc')
pietrainXduroc = sorted2DT(DF3,80,70).copy()
all_count(pietrainXduroc,PIETRAINxDUROC)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,Pietrain X Duroc,244,382,73,11.54,11.0,4.53,14.14,385.0,181.0


In [58]:
krzyzowkiDF = pd.concat([WBPxWBP,WBPxPBZ,PBZxPBZ,PBZxWBP,DUROCxDUROC,DUROCxPIETRAIN,PIETRAINxPIETRAIN,PIETRAINxDUROC])

In [60]:
krzyzowkiDF.set_index('Rasa',inplace = True)

In [61]:
krzyzowkiDF

,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
Rasa,,,,,,,,,
WBP x WBP,1117,1647,383,12.50,11.55,7.08,15.02,372.0,167.0
WBP x PBZ,1077,1668,353,12.46,11.75,5.45,14.94,357.0,166.0
PBZ x PBZ,2415,3860,899,12.63,11.61,7.48,15.32,356.0,173.0
PBZ x WBP,904,1272,386,12.70,12.06,4.81,15.21,367.0,174.0
Duroc x Duroc,135,169,27,11.91,10.87,8.32,13.85,362.0,181.0
Duroc x Pietrain,264,384,76,11.85,10.80,8.19,13.59,374.0,180.0
Pietrain x Pietrain,108,135,22,11.67,10.64,8.24,14.00,374.0,178.0
Pietrain X Duroc,244,382,73,11.54,11.00,4.53,14.14,385.0,181.0
